We would be building a chatbot utilising nltk toolkit and pytorch.

implemented from this video tutorial given here : https://www.youtube.com/watch?v=RpWeNzfSUHw

### Getting the intents.json file

In [30]:
!wget https://raw.githubusercontent.com/python-engineer/pytorch-chatbot/master/intents.json

--2021-05-08 07:40:05--  https://raw.githubusercontent.com/python-engineer/pytorch-chatbot/master/intents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2015 (2.0K) [text/plain]
Saving to: ‘intents.json.1’

intents.json.1      100%[===================>]   1.97K  --.-KB/s    in 0s      

2021-05-08 07:40:05 (27.2 MB/s) - ‘intents.json.1’ saved [2015/2015]



In [31]:
!dir

__notebook_source__.ipynb  data.pth  intents.json  intents.json.1


### nltk_utils.py

In [32]:
import nltk
nltk.download('punkt')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [34]:
def stem(word):
    return stemmer.stem(word.lower())

In [35]:
import numpy as np 

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    
    return bag

In [36]:
# testing 

a = "How long does shipping take?"
print(a)
w = tokenize(a)
print(w)
for b in w:
    print(stem(b))

s = ['How', 'long']

bag = bag_of_words(s,w)
print(bag)


How long does shipping take?
['How', 'long', 'does', 'shipping', 'take', '?']
how
long
doe
ship
take
?
[0. 1. 0. 0. 0. 0.]


### train.py

In [37]:
import json

In [38]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [39]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [40]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']
print(all_words)

['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'items', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'items', 'are', 'there', '?', 'What', 'do', 'you', 'sell', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Paypal', '?', 'Are', 'you', 'cash', 'only', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Do', 'you', 'know', 'a', 'joke', '?']


In [41]:
all_words = [stem(w) for w in all_words if w not in ignore_words]
print(all_words)

['hi', 'hey', 'how', 'are', 'you', 'is', 'anyon', 'there', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodby', 'thank', 'thank', 'you', 'that', "'s", 'help', 'thank', "'s", 'a', 'lot', 'which', 'item', 'do', 'you', 'have', 'what', 'kind', 'of', 'item', 'are', 'there', 'what', 'do', 'you', 'sell', 'do', 'you', 'take', 'credit', 'card', 'do', 'you', 'accept', 'mastercard', 'can', 'i', 'pay', 'with', 'paypal', 'are', 'you', 'cash', 'onli', 'how', 'long', 'doe', 'deliveri', 'take', 'how', 'long', 'doe', 'ship', 'take', 'when', 'do', 'i', 'get', 'my', 'deliveri', 'tell', 'me', 'a', 'joke', 'tell', 'me', 'someth', 'funni', 'do', 'you', 'know', 'a', 'joke']


In [42]:
all_words = sorted(set(all_words))
tags = sorted(set(tags))
print(tags)

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


creating a bag of words

In [43]:
X_train = []
y_train = []
for(pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    
    label = tags.index(tag)
    y_train.append(label)

converting to numpy array

In [44]:

X_train = np.array(X_train)
y_train = np.array(y_train)


In [45]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

implementing the training loop

In [46]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
    
    def __getitem__(self, index ):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
 

In [47]:
batch_size = 8

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)


### model.py

In [48]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation no softmax
        return out

In [49]:
hidden_size = 8
output_size = len(tags)

input_size = len(X_train[0])



In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [51]:
model = NeuralNet(input_size, hidden_size, output_size )

In [52]:
learning_rate = 0.001
num_epochs = 1000

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [53]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words
        labels = labels
        
        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if ((epoch +1)% 100 == 0):
        print('epoch: ', epoch + 1 , '/', num_epochs, ' loss: ', loss.item())

print("final loss: ", loss.item())

epoch:  100 / 1000  loss:  1.3243472576141357
epoch:  200 / 1000  loss:  0.11075551807880402
epoch:  300 / 1000  loss:  0.12341194599866867
epoch:  400 / 1000  loss:  0.024452412500977516
epoch:  500 / 1000  loss:  0.004373390693217516
epoch:  600 / 1000  loss:  0.0042567807249724865
epoch:  700 / 1000  loss:  0.0015149821992963552
epoch:  800 / 1000  loss:  0.0017542962450534105
epoch:  900 / 1000  loss:  0.001718759536743164
epoch:  1000 / 1000  loss:  0.0006780103431083262
final loss:  0.0006780103431083262


lets save the model

In [54]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size" : hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print("Training complete and file save to", FILE)

Training complete and file save to data.pth


### chat.py

In [56]:
import random

with open('intents.json', 'r' ) as f:
    intents = json.load(f)

FILE = "data.pth"

data_loaded = torch.load(FILE)

input_size_l = data_loaded["input_size"]
hidden_size_l = data_loaded["hidden_size"]
output_size_l = data_loaded["output_size"]

all_words = data_loaded["all_words"]
tags = data_loaded["tags"]
model_state = data_loaded["model_state"]

model_l = NeuralNet(input_size_l, hidden_size_l, output_size_l)
model_l.load_state_dict(model_state)

model.eval()


NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [61]:
bot_name = "Samus"
print("let chat! type quit to exit")

while True:
    sentence = input('You : ' )
    if sentence == "quit":
        break
    
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    output = model_l(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    
    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(bot_name, ":",random.choice(intent["responses"]))
    else:
        print(bot_name, ":", "Well, I do not understand.")

let chat! type quit to exit


You :  Hi


Samus : Hi there, what can I do for you?


You :  How can I pay


Samus : Well, I do not understand.


You :  quit
